<h1> DS_7331 Machine Learning - Lab 1 - Group 4 </h1>

Paticia Attah
Linda Eliasen
Senthil Kumar

<h2> BUSINESS UNDERSTANDING </h2>

The H-1B Dataset selected for this project contains data from employer’s Labor Condition Application and the case certification determinations processed by the Office of Foreign Labor Certification (OFLC) where the date of the determination was issues on or after October 1, 2016 and on or before June 30, 2017.

The Labor Condition Application (LCA) is a document that a perspective H-1B employer files with U.S. Department of Labor Employment and Training Administration (DOLETA) when it seeks to employ non-immigrant workers at a specific job occupation in an area of intended employment for not more than three years. 

Our goal for this project is to predict the case status of an application submitted by the employer to hire non-immigrant workers under the H-1B visa program. Employer can hire non-immigrant workers only after their LCA petition is approved. The approved LCA petition is then submitted as part of the Petition for a Non-immigrant Worker application
for work authorizations for H-1B visa status.

We want to uncover insights that can help employers understand the process of getting their LCA approved. We will use WEKA software to run data mining algorithms to understand the relationship between attributes and the target variable.

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import chart_studio.plotly as py
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import matplotlib.pyplot as plt
init_notebook_mode(connected=True)
from plotly import tools
import warnings
warnings.filterwarnings("ignore")

In [22]:
df = pd.read_csv("/home/jovyan/work/extended_case_1_student/cleaned_Visa_Data.csv")

In [23]:
df.head()

,Unnamed: 0,CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,EMPLOYER_NAME,EMPLOYER_CITY,EMPLOYER_STATE,AGENT_REPRESENTING_EMPLOYER,JOB_TITLE,...,ANNUALIZED_WAGE_RATE_OF_PAY,WAGE_RATE_OF_PAY,WAGE_UNIT_OF_PAY,H1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,LABOR_CON_AGREE,WORKSITE_CITY,WORKSITE_STATE,DAYS_PROCESS
0,35,I-200-17170-983629,CERTIFIED,2017-06-19,2017-06-24,WIPRO LIMITED,EAST BRUNSWICK,NJ,N,PROJECT MANAGER,...,86715.0,86715.0,Year,Y,N,Y,N,ANCHORAGE,AK,5
1,36,I-200-17172-419302,CERTIFIED,2017-06-21,2017-06-27,WIPRO LIMITED,EAST BRUNSWICK,NJ,N,VLSI ARCHITECT-L1,...,86694.0,86694.0,Year,Y,N,Y,N,ANCHORAGE,AK,6
2,38,I-200-17111-126952,CERTIFIED,2017-04-21,2017-04-27,WIPRO LIMITED,EAST BRUNSWICK,NJ,N,PROGRAMMER ANALYST,...,82826.0,82826.0,Year,Y,N,Y,N,ANCHORAGE,AK,6
3,63,I-200-17136-596227,CERTIFIED,2017-05-16,2017-05-22,"TECH MAHINDRA (AMERICAS),INC.",PLANO,TX,N,QUALITY ENGINEER,...,72551.0,72551.0,Year,Y,N,Y,Y,ANCHORAGE,AK,6
4,64,I-200-17024-685705,CERTIFIED,2017-01-24,2017-01-30,"TECH MAHINDRA (AMERICAS),INC.",PLANO,TX,N,QUALITY TEST ENGINEER,...,72551.0,72551.0,Year,Y,N,Y,Y,ANCHORAGE,AK,6


In [26]:
df.apply(lambda x:len(x.unique()))

Unnamed: 0                     86338
CASE_NUMBER                    86338
CASE_STATUS                        3
CASE_SUBMITTED                   275
DECISION_DATE                    202
EMPLOYER_NAME                   3507
EMPLOYER_CITY                    801
EMPLOYER_STATE                    48
AGENT_REPRESENTING_EMPLOYER        2
JOB_TITLE                       6540
SOC_CODE                         224
SOC_NAME                         234
SOC_GROUP_Code                    20
SOC_GROUP                         20
TOTAL_WORKERS                     28
NEW_EMPLOYMENT                    27
CONTINUED_EMPLOYMENT              13
CHANGE_PREVIOUS_EMPLOYMENT        12
NEW_CONCURRENT_EMPLOYMENT          5
ANNUALIZED_PREVAILING_WAGE      3999
PREVAILING_WAGE                 4216
PW_UNIT_OF_PAY                     5
PW_WAGE_LEVEL                      1
ANNUALIZED_WAGE_RATE_OF_PAY     5626
WAGE_RATE_OF_PAY                5828
WAGE_UNIT_OF_PAY                   5
H1B_DEPENDENT                      2
W

In [27]:
df.isnull().sum()[df.isnull().sum() > 0]

Series([], dtype: int64)

In [29]:
for i in df.index:   
        if df.loc[i,'PW_UNIT_OF_PAY'] == 'Month':
            df.loc[i,'PREVAILING_WAGE'] = df.loc[i,'PREVAILING_WAGE'] * 12
        if df.loc[i,'PW_UNIT_OF_PAY'] == 'Week':
            df.loc[i,'PREVAILING_WAGE'] = df.loc[i,'PREVAILING_WAGE'] * 48
        if df.loc[i,'PW_UNIT_OF_PAY'] == 'Bi-Weekly':
            df.loc[i,'PREVAILING_WAGE'] = df.loc[i,'PREVAILING_WAGE'] * 24

In [30]:
df.PW_UNIT_OF_PAY.replace(['Bi-Weekly','Month','Week'],['Year','Year','Year'], inplace=True)

In [31]:
df.head()

,Unnamed: 0,CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,EMPLOYER_NAME,EMPLOYER_CITY,EMPLOYER_STATE,AGENT_REPRESENTING_EMPLOYER,JOB_TITLE,...,ANNUALIZED_WAGE_RATE_OF_PAY,WAGE_RATE_OF_PAY,WAGE_UNIT_OF_PAY,H1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,LABOR_CON_AGREE,WORKSITE_CITY,WORKSITE_STATE,DAYS_PROCESS
0,35,I-200-17170-983629,CERTIFIED,2017-06-19,2017-06-24,WIPRO LIMITED,EAST BRUNSWICK,NJ,N,PROJECT MANAGER,...,86715.0,86715.0,Year,Y,N,Y,N,ANCHORAGE,AK,5
1,36,I-200-17172-419302,CERTIFIED,2017-06-21,2017-06-27,WIPRO LIMITED,EAST BRUNSWICK,NJ,N,VLSI ARCHITECT-L1,...,86694.0,86694.0,Year,Y,N,Y,N,ANCHORAGE,AK,6
2,38,I-200-17111-126952,CERTIFIED,2017-04-21,2017-04-27,WIPRO LIMITED,EAST BRUNSWICK,NJ,N,PROGRAMMER ANALYST,...,82826.0,82826.0,Year,Y,N,Y,N,ANCHORAGE,AK,6
3,63,I-200-17136-596227,CERTIFIED,2017-05-16,2017-05-22,"TECH MAHINDRA (AMERICAS),INC.",PLANO,TX,N,QUALITY ENGINEER,...,72551.0,72551.0,Year,Y,N,Y,Y,ANCHORAGE,AK,6
4,64,I-200-17024-685705,CERTIFIED,2017-01-24,2017-01-30,"TECH MAHINDRA (AMERICAS),INC.",PLANO,TX,N,QUALITY TEST ENGINEER,...,72551.0,72551.0,Year,Y,N,Y,Y,ANCHORAGE,AK,6


In [38]:
df['countvar'] = 1

In [39]:
dftop = df.groupby('EMPLOYER_NAME',as_index=False).count()

In [40]:
dftop = dftop.sort_values('countvar',ascending= False)[['EMPLOYER_NAME','countvar']]

In [42]:
dfempst = df.groupby('EMPLOYER_STATE',as_index=False).count()[['EMPLOYER_STATE','countvar']].sort_values('countvar',ascending=False)

In [43]:
t1 = go.Bar(x=dfempst.EMPLOYER_STATE.values,y=dfempst.countvar.values,name='Employerstate')
layout = go.Layout(dict(title= "NUMBER OF APPLICATIONS PER STATE",xaxis=dict(title="STATES"),yaxis=dict(title="Num of applications")))
data = [t1]
fig =go.Figure(data,layout)
iplot(fig)

In [44]:
data=[dict(
    type='choropleth',
    locations = dfempst.EMPLOYER_STATE,
    z = dfempst.countvar,
    locationmode = 'USA-states',marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Number of applications")
)]
layout= dict(title="2011-2018 H1B VISA APPLICATIONS ( EMPLOYER STATE)",geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
fig = dict( data=data, layout=layout )
iplot(fig)

In [45]:
dum = df[["EMPLOYER_STATE","JOB_TITLE"]]
dum = dum.groupby(["EMPLOYER_STATE","JOB_TITLE"]).size().reset_index()
dum.columns = ['EMPLOYER_STATE', 'JOB_TITLE', "COUNT"]
dum = dum.groupby(['EMPLOYER_STATE', 'JOB_TITLE']).agg({'COUNT':sum})
dum = dum['COUNT'].groupby(level=0, group_keys=False)
dum = dum.apply(lambda x: x.sort_values(ascending=False).head(1))
dum = pd.DataFrame(dum).reset_index()

In [46]:
data=[dict(
    type='choropleth',
    locations = dum.EMPLOYER_STATE,
    z = dum.COUNT,
    locationmode = 'USA-states',
    text = dum.JOB_TITLE,
    marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Number of application")
)]
layout= dict(title="Top job title in the state",geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
fig = dict( data=data, layout=layout )
iplot(fig)

In [47]:
dum = df.groupby('EMPLOYER_STATE',as_index=False).mean()[['EMPLOYER_STATE','PREVAILING_WAGE']]

In [48]:
data=[dict(
    type='choropleth',
    locations = dum.EMPLOYER_STATE,
    z = dum.PREVAILING_WAGE,
    locationmode = 'USA-states',
    marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Avg salary in USD")
)]
layout= dict(title="Average salaries per state",geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
fig = dict( data=data, layout=layout )
iplot(fig)